<a href="https://colab.research.google.com/github/bmreiniger/datascience.stackexchange/blob/master/SE77827945_xgb_min_max_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_breast_cancer
from xgboost import XGBClassifier

In [23]:
X, y = load_breast_cancer(return_X_y=True, as_frame=True)
model = XGBClassifier(n_estimators=100, eta=0.001) #, max_depth=2, min_child_weight=20, reg_lambda=20)
model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.001, eval_metric=None,
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...)

In [24]:
# courtesy https://stackoverflow.com/q/37677496/10495893
mf = model.get_booster().trees_to_dataframe()
mf

,Tree,Node,ID,Feature,Split,Yes,No,Missing,Gain,Cover,Category
0,0,0,0-0,worst radius,16.8200,0-1,0-2,0-2,387.436768,133.397995,NaN
1,0,1,0-1,worst concave points,0.1357,0-3,0-4,0-4,57.568939,88.853851,NaN
2,0,2,0-2,worst texture,19.5900,0-5,0-6,0-6,16.615128,44.544148,NaN
3,0,3,0-3,radius error,0.6450,0-7,0-8,0-8,3.126236,77.835037,NaN
4,0,4,0-4,worst texture,25.5800,0-9,0-10,0-10,19.462505,11.018815,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2121,99,20,99-20,Leaf,NaN,NaN,NaN,NaN,-0.001584,1.944909,NaN
2122,99,21,99-21,Leaf,NaN,NaN,NaN,NaN,0.000559,1.591984,NaN
2123,99,22,99-22,Leaf,NaN,NaN,NaN,NaN,0.001482,69.217522,NaN
2124,99,23,99-23,Leaf,NaN,NaN,NaN,NaN,-0.000468,1.188841,NaN


In [25]:
# (It's a little weird, but leaf scores seem to be in the Gain column here.)

# This computes the smallest and largest score each tree produces:
leaf_bounds = mf.loc[mf['Feature'] == 'Leaf'].groupby('Tree')['Gain'].agg(['min', 'max'])
leaf_bounds.head()

,min,max
Tree,,
0,-0.002599,0.001541
1,-0.002595,0.001553
2,-0.002590,0.001540
3,-0.002586,0.001551
4,-0.002582,0.001538


So we get a bound on the sum of leaf scores for any observation:
it can at worst reach the extremal-score leaf in every tree.

(This doesn't have to be achieved: perhaps to reach the min-leaf of tree 1
requires feature1 < 0, and to reach the min-leaf of tree 2 requires feature1 > 5.  We'll test that at the end.)


In [26]:
leaf_sum_bounds = leaf_bounds.sum()
leaf_sum_lb = leaf_sum_bounds['min']
leaf_sum_ub = leaf_sum_bounds['max']
leaf_sum_lb, leaf_sum_ub

(-0.24103466091, 0.15116194701)

...but the sum of leaf scores isn't actually the prediction,
we need to take into consideration the base score and the learning rate.

This method to extract this information is courtesy of:
https://discuss.xgboost.ai/t/how-to-get-base-score-from-trained-booster/3192/3

In [27]:
import json
params = json.loads(model.get_booster().save_config())
base_score = float(
    params["learner"]["learner_model_param"]["base_score"]
)
learning_rate = float(
    params["learner"]["gradient_booster"]["tree_train_param"]["eta"]
)
base_score, learning_rate

(0.6247282, 0.00100000005)

In [28]:
y.mean(), base_score

(0.6274165202108963, 0.6247282)

(Why don't those match???  It's not just the link function, applying it or its inverse doesn't help...)

In [29]:
from scipy.special import expit, logit
lb = expit(logit(base_score) + leaf_sum_lb)
ub = expit(logit(base_score) + leaf_sum_ub)
lb, ub

(0.5667568973344917, 0.6594463758204882)

Those above are the theoretical bounds we've figured out.

Below, the extremal values from the training set:

In [30]:
y_pred = model.predict_proba(X)[:, 1]
y_pred.min(), y_pred.max()

(0.56675696, 0.6594464)

Despite the difference between y.mean() and base_score, it looks like the bounds agree with the predictions (up to rounding) for all the models I've tried.

With a small ensemble (`n_estimators=5`) they match to the displayed precision.

When I try with the default parameters (100 trees), the predictions get pushed very close to 0 & 1, so it's not as clear whether they agree up to rounding.

Setting `n_estimators=100` but `eta=0.001`, I get a very small difference in the lower bound and minimum predictions; is that rounding, or does the smallest prediction not come from a row that hits the minimum leaf in every tree??  We can test that directly!

In [31]:
from xgboost import DMatrix
import numpy as np

idx = y_pred.argmin()  # minimum prediction
row = X.iloc[idx:idx+1]
dm = DMatrix(row)

leaves = model.get_booster().predict(dm, pred_leaf=True).ravel()  # pred_leaf=True is finding the leaf index the row gets bucketed to
leaf_scores = []
for i, leaf in enumerate(leaves):
    leaf_scores.append(mf.loc[(mf['Tree'] == i) & (mf['Node'] == leaf), 'Gain'].item())
leaf_scores = np.array(leaf_scores)

np.array_equal(
    mf.loc[mf['Feature'] == 'Leaf'].groupby('Tree')['Gain'].agg('min').values,
    leaf_scores,
)

True

guess it's rounding :)

I can't get a model with different theoretical bounds and training predictions.  I've tried making the model more-underfit using a few different hyperparameters.  It might need a bigger / more complex dataset, or cleverly constructing a dataset?...